In [4]:
from CompactFIPS202 import *
import binascii
import os
from hashlib import sha3_256, sha3_512, shake_128, shake_256
from kyber import Kyber512, Kyber768, Kyber1024
from aes256_ctr_drbg import AES256_CTR_DRBG
import itertools
import utils
from bitstring import Bits
from fixedpoint import FixedPoint

os.system('rm -rf ./vec')

def parse_kat_data(data):
    parsed_data = {}
    count_blocks = data.split('\n\n')
    for block in count_blocks[1:-1]:
        block_data = block.split('\n')
        count, seed, pk, sk, ct, ss = [line.split(" = ")[-1] for line in block_data]
        parsed_data[count] = {
            "seed": bytes.fromhex(seed),
            "pk": bytes.fromhex(pk),
            "sk": bytes.fromhex(sk),
            "ct": bytes.fromhex(ct),
            "ss": bytes.fromhex(ss),   
        }
    return parsed_data

In [5]:
with open("assets/PQCkemKAT_1632.rsp") as f:
    kat_data = f.read()
    parsed_data = parse_kat_data(kat_data)

data = parsed_data['0']
seed, pk, sk, ct, ss = data.values()
        
# Seed DRBG with KAT seed
Kyber512.set_drbg_seed(seed)
# Assert keygen matches
_pk, _sk = Kyber512.keygen()

# Assert encapsulation matches
_ct, _ss = Kyber512.enc(_pk)

# Assert decapsulation matches
__ss = Kyber512.dec(ct, sk)

-------------------------
Kyber Key Generation
-------------------------
-------------------------
Kyber Encryption
-------------------------
:::Decompressing u
[DECOMPRESS] COEFF_ORIG   : [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1]
[DECOMPRESS] D/Q          : 1/3329
[DECOMP

In [6]:
(pk == _pk) == (sk == _sk) == (ct == _ct) == (ss == _ss)

True

In [7]:
Kyber512.R.cbd_input_bytes

b'm\x98\xe3\xa6hz\x1e\xa0\xf8\xe7\xe2\x0b\x0b\xc6\x92t\xaf\xd5\xc5+*\xb9F\xbb\xa8BBR\x93\n\x98y\x0c\xb7+}p\x1a\x18\xb7l\xe2#\xd2\xbb\xa0.\xbbo\xca\xe4\x82\xf9\x00\x05\x82/u\xa6\xbd\xa2]F\xa7\xa7\xfd\xab\xc0HT\n\xafY0\xd2ViH}h\x8b\xa4\xb5\x9b\xa0\xb4\xd5\xe0\xc0K\xa9\xd9WU\x81F\x11E\xe6\xf6\x95~\xb9}\xbfM0\xf9\xe0\x7f\xee\x15=\xb3i\xbe\xe1\xbc\xf5/\x9fA\xe9)z\x15\x1c6'

In [8]:
int.from_bytes(Kyber512.R.cbd_input_bytes)

76961787726721717185743086793776129555753488705600050978100509952925563418804137206042903186472020713167622928246417569670876646104736080018509078121785469511704207335593646502772006212260109437457761327505238322446707092243050096409058848007552273633535859835778791864986071398948929103352012325597778091062

In [9]:
format(Kyber512.R.cbd_input_bytes[0], '08b')[::-1]

'10110110'

In [10]:
a = Bits(int=-1, length=4).hex
a

'f'

In [11]:
Kyber512.R.cbd_coefficients[0:7]

[-1, 0, -1, 0, 2, -1, 0]

In [12]:
test = int(''.join(Bits(int=x, length=3).bin for x in Kyber512.R.cbd_coefficients), 2)
test

1379815459959135610830526476087135324366633471742828041500658720353569895110810220672444945926538379163903179333179126586549286914266357358770558081720749945850335746292028301873611507697264927320785394348067908251632314078969732162

In [13]:
Kyber512._generate_error_vector_v

[                                                                                                                                                                                                             -1 + -1*x^3 + -2*x^4 + x^8 + -1*x^9 + x^10 + -1*x^11 + x^12 + 2*x^14 + -1*x^16 + x^17 + -1*x^18 + x^20 + x^22 + x^23 + x^25 + -1*x^27 + x^29 + -1*x^31 + x^32 + -1*x^34 + -1*x^37 + 2*x^38 + -1*x^39 + -2*x^41 + -1*x^43 + x^44 + 2*x^46 + -2*x^47 + -1*x^49 + -1*x^52 + -1*x^55 + x^56 + -1*x^57 + x^59 + x^60 + -1*x^61 + -1*x^62 + 2*x^63 + -1*x^64 + x^67 + x^71 + x^72 + -2*x^73 + x^74 + x^75 + -1*x^77 + x^78 + -2*x^80 + x^83 + 2*x^84 + 2*x^87 + x^89 + x^90 + x^93 + x^94 + -1*x^96 + -2*x^97 + 2*x^99 + -1*x^100 + -1*x^103 + -1*x^106 + 2*x^107 + -1*x^111 + -1*x^113 + 2*x^117 + x^119 + -2*x^120 + x^124 + x^125 + -1*x^126 + 2*x^130 + -2*x^132 + -2*x^136 + x^138 + 2*x^139 + x^141 + -1*x^142 + -1*x^148 + 2*x^149 + -1*x^151 + x^153 + -2*x^155 + x^157 + -2*x^159 + x^160 + x^161 + -1*x^162 + x^163 + 

In [14]:
Kyber512._generate_error_vector_N

4

In [87]:
a = 2**1/3329
a

0.0006007810153199159

In [88]:
fixed_num = FixedPoint(a, signed=False, m=0, n=24, str_base=16)
float(fixed_num), fixed_num.qformat

(0.0006007552146911621, 'UQ0.24')

In [84]:
fixed_num

FixedPoint('0x275f', signed=0, m=0, n=24, overflow='clamp', rounding='nearest', overflow_alert='error', mismatch_alert='warning', implicit_cast_alert='warning', str_base=16)

In [86]:
float(fixed_num) * 3328

1.9993133544921875